In [4]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO
import torch
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.nn as nn
import os
import json


# Cuda check and visualize data

In [18]:
print("PyTorch CUDA version:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
print("CUDA devices:", torch.cuda.device_count())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No device")

PyTorch CUDA version: 12.4
CUDA available: True
CUDA devices: 1
CUDA device name: NVIDIA GeForce GTX 1650


In [19]:
def visualize_image_with_bboxes(image_path, annotation_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    with open(annotation_path, 'r') as file:
        annotations = file.readlines()

    height, width, _ = image.shape

    for annotation in annotations:
        values = annotation.strip().split()
        class_id = int(values[0])
        cx, cy, w, h = map(float, values[1:])

        x_min = int((cx - w / 2) * width)
        y_min = int((cy - h / 2) * height)
        x_max = int((cx + w / 2) * width)
        y_max = int((cy + h / 2) * height)

        color = (255, 255, 255)
        if class_id == 0:
            color = (255, 0, 0)
        elif class_id == 1:
            color = (255, 255, 0)

        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 12)

    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

visualize_image_with_bboxes("datasets/cocoa_diseases/images/train/Fito1.jpg", "datasets/cocoa_diseases/labels/train/Fito1.txt")

<Figure size 1000x1000 with 1 Axes>

# Regular YOLO

In [ ]:
model = YOLO("yolov11x")

train_results = model.train(
    data="datasets/cocoa_diseases/cocoa_dataset.yaml",
    epochs=100,
    project="models/yolo11",
    imgsz=640,
    batch=4,  
    device="cuda", 
    workers=5  
)

model.save("models/model_files/yolo11_cocoa_trained.pt")

# Pruning setup 

In [5]:
def prune_model(pytorch_model, amount):
    for name, module in pytorch_model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name="weight", amount=amount)
            prune.remove(module, "weight")
            module.weight = nn.Parameter(module.weight.to_sparse())
    return pytorch_model

# Pre-train pruning YOLO

## No coco retrain 

In [ ]:
model = YOLO("yolo11x.pt")
model.model = prune_model(model.model, 0.05)

train_results = model.train(
    data="datasets/cocoa_diseases/cocoa_dataset.yaml",
    epochs=100,
    project="models/yolo11_pruned_no_retrain",
    imgsz=640,
    batch=4,  
    device="cuda", 
    workers=5  
)

model.save("models/model_files/yolo11_pruned_no_retrain.pt")

## Retrain on coco

In [ ]:
model = YOLO("yolo11x.pt")
model.model = prune_model(model.model, 0.05)

# Retrain on coco8 before training on cocoa dataset
model.train(
    data="datasets/coco8/coco8_dataset.yaml",
    epochs=5,                     
    imgsz=640,
    batch=4,
    device="cuda",
    workers=1             
)

train_results = model.train(
    data="datasets/cocoa_diseases/cocoa_dataset.yaml",
    epochs=100,
    project="models/yolo11_pruned_retrain",
    imgsz=640,
    batch=4,  
    device="cuda", 
    workers=5  
)

model.save("models/model_files/yolo11_pruned_retrain.pt")

# Post-train pruning YOLO

In [ ]:
model = YOLO("models/model_files/yolo11_cocoa_trained.pt")
model.model = prune_model(model.model, 0.05)

train_results = model.train(
    data="datasets/cocoa_diseases/cocoa_dataset.yaml",
    epochs=10,
    patience=5,
    project="models/yolo11_post_pruned_retrain",
    imgsz=640,
    batch=4,  
    device="cuda", 
    workers=5  
)

model.save("models/model_files/yolo11_post_pruned_retrain.pt")

# Testing

In [28]:
model_results = {}

for filename in os.listdir("models/model_files"):
    model = YOLO(f"models/model_files/{filename}")
    results = model.val(data="datasets/cocoa_diseases/cocoa_dataset.yaml")
    
    metrics = {
        "mAP50": results.box.map50,                
        "mAP50-95":results.box.map,   
        "inference_time (ms)": results.speed["inference"]  
    }
    
    model_results[os.path.splitext(filename)[0]] = metrics

output_path = "model_validation_results.json"
with open(output_path, "w") as f:
    json.dump(model_results, f, indent=4)

Ultralytics 8.3.58  Python-3.11.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11x summary (fused): 464 layers, 56,830,489 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning C:\Users\danie\Documents\Programming\Uni\ML2\datasets\cocoa_diseases\labels\val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:20<00:00,  5.05s/it]


                   all         62        289      0.725       0.65      0.731      0.506
          phytophthora         30         39      0.747      0.564      0.675      0.498
               monilia         23         30      0.652      0.767      0.756      0.562
               healthy         52        220      0.777      0.618       0.76      0.457
Speed: 2.2ms preprocess, 78.6ms inference, 0.0ms loss, 4.4ms postprocess per image
Results saved to runs\detect\val6
Ultralytics 8.3.58  Python-3.11.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)


val: Scanning C:\Users\danie\Documents\Programming\Uni\ML2\datasets\cocoa_diseases\labels\val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:13<00:00,  3.33s/it]


                   all         62        289      0.648       0.59      0.628      0.418
          phytophthora         30         39      0.594      0.513      0.586      0.423
               monilia         23         30      0.617        0.6      0.567      0.402
               healthy         52        220      0.731      0.656      0.733      0.427
Speed: 2.9ms preprocess, 11.4ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs\detect\val7
Ultralytics 8.3.58  Python-3.11.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)


val: Scanning C:\Users\danie\Documents\Programming\Uni\ML2\datasets\cocoa_diseases\labels\val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:21<00:00,  5.41s/it]


                   all         62        289      0.561      0.547      0.538      0.357
          phytophthora         30         39       0.42      0.513      0.421      0.297
               monilia         23         30      0.562      0.429      0.441      0.331
               healthy         52        220      0.701        0.7      0.752      0.442
Speed: 3.4ms preprocess, 10.4ms inference, 0.0ms loss, 11.0ms postprocess per image
Results saved to runs\detect\val8


In [6]:
# directory = "datasets/cocoa_diseases/images/val/"

# model = YOLO("models/model_files/yolo11.pt")
# results = model.predict([directory+"Sana25.jpg",directory+"Fito7.jpg",directory+"Monilia31.jpg"])

# for result in results:
#     result.show()


0: 640x480 2 healthys, 77.7ms
1: 640x480 2 phytophthoras, 1 monilia, 3 healthys, 77.7ms
2: 640x480 1 phytophthora, 5 monilias, 3 healthys, 77.7ms
Speed: 0.0ms preprocess, 77.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)


In [11]:
# directory = "datasets/cocoa_diseases/images/val/"

# model = YOLO("models/model_files/yolo11_post_pruned_retrain.pt")
# # results = model.predict([directory+"Sana25.jpg",directory+"Fito7.jpg",directory+"Monilia31.jpg"])
# results = model.predict([directory+"Monilia31.jpg"])

# for result in results:
#     result.save("normal_post_pruned_retrained.png")
#     result.show()



0: 640x480 1 phytophthora, 3 monilias, 4 healthys, 67.6ms
Speed: 5.5ms preprocess, 67.6ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 480)
